# Fake News Project

1. Initial EDA
2. Analysis with Binary labels (fake vs. reliable)

    a. Disregarding domain (web source)
        i. Sentiment analysis
        ii. Prediction
        
    b. Sampling with domain in consideration
        i. Prediction
        
    c. Topic Modeling
        
        
3. Analysis with Multi-class labels

    a. Four largest labels - classification
    
    b. All labels - classification

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import collections




from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
from nltk.corpus import sentiwordnet as swn

from textblob import TextBlob

import gensim
from gensim import corpora
from gensim import models

import pyLDAvis
import pyLDAvis.gensim

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.feature_extraction import text 

from xgboost import XGBClassifier
import langid


ModuleNotFoundError: No module named 'langid'

## Initial EDA

In [2]:
df = pd.read_csv('newly_sampled_data.csv')

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


FileNotFoundError: [Errno 2] No such file or directory: 'newly_sampled_data.csv'

In [3]:
df.shape

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df' is not defined

In [4]:
df.type.value_counts().plot(kind='bar', color='springgreen')
plt.xlabel('News Category')
plt.ylabel('Total Count')
plt.title("Counts for News Type", fontsize=25)
plt.savefig('newstypecounts.png', bbox_inches='tight', dpi = 300)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df' is not defined

In [5]:
df.domain.value_counts().nlargest(20).plot(kind='bar', color='springgreen')
plt.xlabel('Domain Category')
plt.ylabel('Total Count')
plt.title("Counts for Domain", fontsize=25)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df' is not defined

In [6]:
valcounts_domain = df.domain.value_counts()
topten_domain = valcounts_domain.head(10)
topten_domain.plot(kind='bar', color='springgreen')
plt.xlabel('News Source', fontsize=15)
plt.ylabel('Total Count', fontsize=15)
plt.title("Top 10 News Sources in Data", fontsize=25)
plt.savefig('top10newssources.png', bbox_inches='tight', dpi = 300)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df' is not defined

In [7]:
nytimes = df.loc[df['domain'] =='nytimes.com'] 
nytimes.type.value_counts().plot(kind='bar', color='goldenrod')
plt.xlabel('News Category')
plt.ylabel('Total Count')
plt.xticks(fontsize=14, rotation=0)
plt.title("'Type' Counts for nytimes.com", fontsize=25)
plt.savefig('nytimes.png', bbox_inches='tight', dpi = 300)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df' is not defined

The dataset defines every document from The New York Times as being 'reliable'.

In [8]:
beforeitsnews = df.loc[df['domain'] =='beforeitsnews.com'] 
beforeitsnews.type.value_counts().plot(kind='bar', color='goldenrod')
plt.xlabel('News Category')
plt.ylabel('Total Count')
plt.xticks(fontsize=14, rotation=0)
plt.title("'Type' Counts for Beforeitsnews.com", fontsize=25)
plt.savefig('beforeitsnews.png', bbox_inches='tight',  dpi = 300)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df' is not defined

Similarly, every single article from beforeitsnews.com is considered to be 'fake' in the dataset. 

In [9]:
dailykos = df.loc[df['domain'] =='dailykos.com'] 
dailykos.type.value_counts().plot(kind='bar', color='goldenrod')
plt.xlabel('News Category')
plt.ylabel('Total Count')
plt.xticks(fontsize=14, rotation=0)
plt.title("'Type' Counts for dailykos.com", fontsize=25)
plt.savefig('dailykos.png', bbox_inches='tight', dpi = 300)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df' is not defined

In [10]:
sputnik = df.loc[df['domain'] =='sputniknews.com'] 
sputnik.type.value_counts().plot(kind='bar', color='goldenrod')
plt.xlabel('News Category')
plt.ylabel('Total Count')
plt.xticks(fontsize=14, rotation=0)
plt.title("'Type' Counts for sputniknews.com", fontsize=25)
plt.savefig('sputnik.png', bbox_inches='tight', dpi = 300)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df' is not defined

In [11]:
express = df.loc[df['domain'] =='express.co.uk'] 
express.type.value_counts().plot(kind='bar', color='goldenrod')
plt.xlabel('News Category')
plt.ylabel('Total Count')
plt.xticks(fontsize=14, rotation=0)
plt.title("'Type' Counts for express.co.uk", fontsize=25)
plt.savefig('express.co.uk.png', bbox_inches='tight', dpi = 300)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df' is not defined

It should be noted that there is inherent bias in this dataset from the labelling of every single article from a given source as being 'rumor', 'junksci', 'bias', or 'political', as opposed to 'reliable', as any given source will contain a wide variety of documents, with varying levels of veracity.

## Binary Labels

## 'Fake' vs. 'Reliable' news in dataset

In [12]:
df = pd.read_csv('complete_new_data.csv')

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


FileNotFoundError: [Errno 2] No such file or directory: 'complete_new_data.csv'

In [13]:
df.shape

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df' is not defined

In [14]:
df.type.value_counts().plot(kind='bar', color='springgreen')
plt.xlabel('Type Category')
plt.ylabel('Total Count')
plt.xticks(fontsize=14, rotation=0)
plt.title("'Type' Counts in Sampled Data", fontsize=25)
plt.savefig('typecounts.png', bbox_inches='tight', dpi = 300)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df' is not defined

10,000 articles labeled 'fake' and '10,000' articles labelled 'reliable' have been sampled from the larger dataset. 

In [15]:
df = df.drop(['tags', 'summary', 'source'], axis=1)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df' is not defined

In [0]:
df.domain.value_counts().plot(kind='pie',labels= None)
plt.text(-0.75, 0.35, 'beforeitsnews.com', dict(size=15))
plt.text(-0.75, -0.6, 'nytimes.com', dict(size=15))
plt.text(0.75, -0.7, 'Other sites', dict(size=15))
plt.title("News Source (domain)", fontsize=25)
plt.savefig('newssourcepie.png', bbox_inches='tight', dpi = 300)

## Sentiment Analysis with VADER and textblob

In [0]:
fake = df[df['type']=='fake']

In [0]:
fake.shape

In [0]:
fakecontent = []
for index, row in fake.iterrows():
    fakecontent.append(row['content'])

In [0]:
print (len(fakecontent))

In [0]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for doc in fakecontent:
    pol_score = sia.polarity_scores(doc)
    pol_score['content'] = doc
    results.append(pol_score)

In [0]:
fakedf = pd.DataFrame.from_records(results)

In [0]:
fakedf.head()

In [0]:
fakedf['label'] = 0
fakedf.loc[fakedf['compound'] > 0.2, 'label'] = 1
fakedf.loc[fakedf['compound'] < -0.2, 'label'] = -1

In [0]:
fakedf.label.value_counts()

In [0]:
print (fakedf['compound'].mean())

In [0]:
print (fakedf['neg'].mean())

In [0]:
print (fakedf['neu'].mean())

In [0]:
print (fakedf['pos'].mean())

In [0]:
def textblob_polarity_score(text):
    blob = TextBlob(text)
    return blob.sentiment[0]

In [0]:
fakedf["textblob_polarity_score"] = fakedf["content"].apply(textblob_polarity_score)

In [0]:
def textblob_subjectivity_score(text):
    blob = TextBlob(text)
    return blob.sentiment[1]

In [0]:
fakedf["textblob_subjectivity_score"] = fakedf["content"].apply(textblob_subjectivity_score)

In [0]:
fakedf['text_blob_polarity_label'] = 0
fakedf.loc[fakedf['textblob_polarity_score'] > 0.2, 'text_blob_polarity_label'] = 1
fakedf.loc[fakedf['textblob_polarity_score'] < -0.2, 'text_blob_polarity_label'] = -1

In [0]:
fakedf['text_blob_subjectivity_label'] = 0
fakedf.loc[fakedf['textblob_subjectivity_score'] > 0.55, 'text_blob_subjectivity_label'] = 1
fakedf.loc[fakedf['textblob_subjectivity_score'] < 0.45, 'text_blob_subjectivity_label'] = -1

In [0]:
fakedf[fakedf['textblob_subjectivity_score']>=.5].count()

In [0]:
fakedf.text_blob_polarity_label.value_counts()

In [0]:
fakedf.text_blob_subjectivity_label.value_counts()

In [0]:
print (fakedf['textblob_polarity_score'].mean())

In [0]:
print (fakedf['textblob_subjectivity_score'].mean())

In [0]:
reliable = df[df['type']=='reliable']

In [0]:
relcontent = []

In [0]:
for index, row in reliable.iterrows():
    relcontent.append(row['content'])

In [0]:
relresults = []
for doc in relcontent:
    pol_score = sia.polarity_scores(doc)
    pol_score['content'] = doc
    relresults.append(pol_score)

In [0]:
reldf = pd.DataFrame.from_records(relresults)



In [0]:
reldf['label'] = 0
reldf.loc[reldf['compound'] > 0.2, 'label'] = 1
reldf.loc[reldf['compound'] < -0.2, 'label'] = -1

In [0]:
reldf.label.value_counts()

In [0]:
print (reldf['compound'].mean())

In [0]:
print (reldf['neg'].mean())

In [0]:
print (reldf['neu'].mean())

In [0]:
print (reldf['pos'].mean())

In [0]:
def textblob_polarity_score(text):
    blob = TextBlob(text)
    return blob.sentiment[0]

In [0]:
reldf["textblob_polarity_score"] = reldf["content"].apply(textblob_polarity_score)

In [0]:
def textblob_subjectivity_score(text):
    blob = TextBlob(text)
    return blob.sentiment[1]

In [0]:
reldf["textblob_subjectivity_score"] = reldf["content"].apply(textblob_subjectivity_score)

In [0]:
reldf['text_blob_polarity_label'] = 0
reldf.loc[reldf['textblob_polarity_score'] > 0.2, 'text_blob_polarity_label'] = 1
reldf.loc[reldf['textblob_polarity_score'] < -0.2, 'text_blob_polarity_label'] = -1

In [0]:
reldf['text_blob_subjectivity_label'] = 0
reldf.loc[reldf['textblob_subjectivity_score'] > 0.55, 'text_blob_subjectivity_label'] = 1
reldf.loc[reldf['textblob_subjectivity_score'] < 0.45, 'text_blob_subjectivity_label'] = -1

In [0]:
reldf[reldf['textblob_subjectivity_score']>=.5].count()

In [0]:
reldf.text_blob_polarity_label.value_counts()

In [0]:
reldf.text_blob_subjectivity_label.value_counts()

In [0]:
print (reldf['textblob_polarity_score'].mean())

In [0]:
print (reldf['textblob_subjectivity_score'].mean())

## Classification (Predictive Analysis)

First, some text cleaning is involved. Various steps are outlined below. 

Tokenize article text.

In [0]:
df.head()

In [0]:
punctuation = string.punctuation

Remove punctuation and stem the words. 

In [0]:
def remove_punc(row):
    tokens = row.split()
    newtokens = [w for w in tokens if w.isalpha()]
    newtokens = [w for w in newtokens if w not in punctuation]
    return newtokens

In [0]:
df['no_punc_content'] = df.content.apply(remove_punc)

In [0]:
ps = PorterStemmer()

def stemmed_content(row):
    content = row['no_punc_content']
    stemmed_list = [ps.stem(word) for word in content]
    return (stemmed_list)

df['stemmed_words'] = df.apply(stemmed_content, axis=1)

df.head()

**Remove stop words**

In [0]:
english_stopwords = set(stopwords.words('english'))


def remove_stop(row):
    tokens = row['stemmed_words']
    newtokens = [w for w in tokens if w not in english_stopwords]
    return newtokens

df['no_stop_stemmed'] = df.apply(remove_stop, axis=1)



df.head()

The following steps create a predictive model using count vectorizer and Multinomial Naive Bayes.  These are in individual steps.  All subsequent predictive models are made using the sklearn pipeline class. 

Rejoin lists of words in the 'no_stop_stemmed' column.

In [0]:
def rejoin(row):
    my_list = row['no_stop_stemmed']
    joined_text = ( " ".join(my_list))
    return joined_text

df['pre_processed_text'] = df.apply(rejoin, axis=1)

df.head()

In [0]:
df.type.unique()

In [0]:
df = df.dropna(subset=['type'])

In [0]:
df.type.unique()

In [0]:
y = df.type

Split the data into training and testing data. 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['pre_processed_text'], y, test_size=0.33, random_state=53)

Create Count Vectorizer.

In [0]:
count_vectorizer = CountVectorizer(stop_words='english')

In [0]:
count_train = count_vectorizer.fit_transform(X_train)

count_test = count_vectorizer.transform(X_test)

In [0]:
print(count_vectorizer.get_feature_names()[:100])

**Train and test the model**

In [0]:
nb_classifier = MultinomialNB()


nb_classifier.fit(count_train, y_train)
pred = nb_classifier.predict(count_test)


score = metrics.accuracy_score(y_test, pred)
print(score)

In [0]:
def show_most_informative_features(vectorizer, clf, n=25):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [0]:
show_most_informative_features(count_vectorizer, nb_classifier)

Many of the most predictive features (on the right hand column above) are linked specifically to the New York Times. Let's extend the stopwords for different results.

The English stopwords included in scikit-learn:

In [0]:
print (text.ENGLISH_STOP_WORDS)

Let's extend this:

In [0]:
additional_sw = ('new', 'york', 'time', 'continu', 'read', 'stori','wa','hi','ha','pleas','like','newslett','view')

In [0]:
all_stop_words = text.ENGLISH_STOP_WORDS.union(additional_sw)


Redo analysis:

In [0]:
count_vectorizer = CountVectorizer(stop_words=all_stop_words)
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)

In [0]:
nb_classifier = MultinomialNB()
nb_classifier.fit(count_train, y_train)
pred = nb_classifier.predict(count_test)
score = metrics.accuracy_score(y_test, pred)
print(score)

In [0]:
show_most_informative_features(count_vectorizer, nb_classifier)

### Classification Using a Pipeline ##

The above steps can also be completed conveniently using a pipeline.

Pipeline with count_vectorizer() using several classifiers. 

In [0]:
classifiers = [MultinomialNB(),LinearSVC(), XGBClassifier()]

In [0]:
class pre_process(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def transform(self, X, y=None):
        
        def proc(z):
            text = z.split()
            text =  [w for w in text if w.isalpha()]
            text = [w for w in text if w not in punctuation]
            text = [ps.stem(word) for word in text]
            text = [w for w in text if w not in english_stopwords]
            text = " ".join(text)
            return text
        z = X.apply(proc)        
        return z
    def fit(self, df, y=None):
        return self

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['pre_processed_text'], y, test_size=0.33, random_state=53)

In [0]:
for classifier in classifiers:
    tfidf_pineline = Pipeline([
        #('preproc', pre_process()),
              ('cv', CountVectorizer(stop_words='english')),
              ('clf',  classifier  )])
    tfidf_pineline.fit(X_train,y_train)
    y_pred = tfidf_pineline.predict(X_test)
    print ("\n\nName of classifier being used with tfidf vectorizer: \n\n" + str(classifier) + 
           "\n\nClassification Report : \n\n"+ str(metrics.classification_report(y_test,y_pred))  + "\n\n Accuracy of classifier : " + str(tfidf_pineline.score(X_test,y_test)))

**Same but with tf-idf**

Use sklearn pipeline to create model using tf-idf vectorizer.

In [0]:
for classifier in classifiers:
    tfidf_pineline = Pipeline([
        #('preproc', pre_process()),
              ('cv', TfidfVectorizer()),
              ('clf',  classifier  )])
    tfidf_pineline.fit(X_train,y_train)
    y_pred = tfidf_pineline.predict(X_test)
    print ("\n\nName of classifier being used with tfidf vectorizer: \n\n" + str(classifier) + 
           "\n\nClassification Report : \n\n"+ str(metrics.classification_report(y_test,y_pred))  + "\n\n Accuracy of classifier : " + str(tfidf_pineline.score(X_test,y_test)))

As noted above, these steps can also be completed without using a pipeline. Steps are repeated below in order to make use of the sbow_most_informative_features() function with the Linear SVC classifier, which was the most accurate.

In [0]:
tfidf_vect = TfidfVectorizer()

X_tfidf_train = tfidf_vect.fit_transform(X_train)

X_tfidf_test = tfidf_vect.transform(X_test)

clf = LinearSVC()

clf.fit(X_tfidf_train, y_train)
pred = clf.predict(X_tfidf_test)

score = metrics.accuracy_score(y_test, pred)
print(score)

In [0]:
show_most_informative_features(tfidf_vect, nb_classifier)

**Tf-idf with bigrams**

In [0]:
classifiers = [MultinomialNB(),LinearSVC(), XGBClassifier()]

In [0]:
for classifier in classifiers:
    tfidf_pineline_w_bigrams = Pipeline([
        #('preproc', pre_process()),
              ('cv', TfidfVectorizer(ngram_range=(2,2))),
              ('clf',   classifier)])
    tfidf_pineline_w_bigrams.fit(X_train,y_train)
    y_pred = tfidf_pineline_w_bigrams.predict(X_test)
    print ("\n\nName of classifier being used with tfidf vectorizer: \n\n" + str(classifier) + 
           "\n\nClassification Report : \n\n"+ str(metrics.classification_report(y_test,y_pred))  + "\n\n Accuracy of classifier : " + str(tfidf_pineline_w_bigrams.score(X_test,y_test)))

In [0]:
for classifier in classifiers:
    tfidf_pineline_w_trigrams = Pipeline([
              ('cv', TfidfVectorizer(ngram_range=(3,3))),
              ('clf',   classifier)])
    tfidf_pineline_w_trigrams.fit(X_train,y_train)
    y_pred = tfidf_pineline_w_trigrams.predict(X_test)
    print ("\n\nName of classifier being used with tfidf vectorizer: \n\n" + str(classifier) + 
           "\n\nClassification Report : \n\n"+ str(metrics.classification_report(y_test,y_pred))  + "\n\n Accuracy of classifier : " + str(tfidf_pineline_w_trigrams.score(X_test,y_test)))

Vectorization with trigrams is significantly less accurate than with bigrams. 

The td-idf vectorizer with SVM was the most accurate.  It is implemented below without the sklearn pipeline to get the most informative features. 

In [0]:
tfidf_vect_bigrams = TfidfVectorizer(ngram_range=(2,2))

In [0]:
X_tfidf__bigrams_train = tfidf_vect_bigrams.fit_transform(X_train)

In [0]:
X_tfidf__bigrams_test = tfidf_vect_bigrams.transform(X_test)

In [0]:
linsvc = LinearSVC()

In [0]:
linsvc.fit(X_tfidf__bigrams_train, y_train)
pred = linsvc.predict(X_tfidf__bigrams_test)

In [0]:
score = metrics.accuracy_score(y_test, pred)
print(score)

In [0]:
show_most_informative_features(tfidf_vect_bigrams, linsvc)

The above results show that bigrams relating directly to the newspaper the New York Times are most predictive.  This is problematic because we want the classifier to predict whether an article is fake or reliable -- not whether it appeared in the New York Times.

There are a few ways this issue can be approached -- we can include more stopwords relating specifically to the New York Times, or we can resample the data to include more sources.

Let's redo this including stop words relating specifically to the New York Times. 

In [0]:
tfidf_vect_bigrams = TfidfVectorizer(ngram_range=(2,2),stop_words=all_stop_words )

In [0]:
X_tfidf__bigrams_train = tfidf_vect_bigrams.fit_transform(X_train)
X_tfidf__bigrams_test = tfidf_vect_bigrams.transform(X_test)

In [0]:
linsvc = LinearSVC()
linsvc.fit(X_tfidf__bigrams_train, y_train)
pred = linsvc.predict(X_tfidf__bigrams_test)

In [0]:
score = metrics.accuracy_score(y_test, pred)
print(score)

In [0]:
show_most_informative_features(tfidf_vect_bigrams, linsvc)

Many of these features are still clearly associated with New York times subscription. Let's try including a few more stopwords and doing the analysis again.

In [0]:
more_sw = ('advertis','subscrib','main','email','thank','sign','verifi','robot','updat','click','select','offer','new', 'york', 'time', 'continu', 'read', 'stori','wa','hi','ha','pleas','like','newslett','view')

all_stop_words = text.ENGLISH_STOP_WORDS.union(more_sw)


In [0]:
tfidf_vect_bigrams = TfidfVectorizer(ngram_range=(2,2),stop_words=all_stop_words )
X_tfidf__bigrams_train = tfidf_vect_bigrams.fit_transform(X_train)
X_tfidf__bigrams_test = tfidf_vect_bigrams.transform(X_test)

In [0]:
linsvc = LinearSVC()
linsvc.fit(X_tfidf__bigrams_train, y_train)
pred = linsvc.predict(X_tfidf__bigrams_test)

In [0]:
score = metrics.accuracy_score(y_test, pred)
print(score)

In [0]:
show_most_informative_features(tfidf_vect_bigrams, linsvc)

While slightly less accurate, some of these features appear to be more relefective of the content of news articles, rather than whether a given article was found in the New York Times. 


Now, let's try the approach of undersampling the data, so that fewer articles from the New York Times are in the dataset. 

### Predictive Analysis with domain considered

In [0]:
df.type.value_counts()

In [0]:
df.domain.value_counts()

NYT and Beforeitsnews are vastly overrepresented in the data. Let's preform undersampling.

Because the unbalanced larger classes are usually undersampled to the number of rows in the smallest represented classes, let's only include observations with domain classes that are represented at least 100 times in the dataset. There are 10 such classes. 

In [0]:
df.domain.value_counts().nlargest(10).plot(kind='bar', color='dodgerblue')
plt.xlabel('Domain')
plt.ylabel('Total Count')
plt.xticks(fontsize=14, rotation=90)
plt.title("'Domain' Counts in Initially Sampled Data", fontsize=25)
plt.savefig('domaincounts.png', bbox_inches='tight', dpi = 300)

In [0]:
df.domain.value_counts().nlargest(10)

NYT and Beforeitsnews are vastly overrepresented in the data. Let's preform undersampling.

In [0]:
newdf =  df[df.groupby("domain")['domain'].transform('size') > 106]


In [0]:
grouped = newdf.groupby('domain', group_keys=False)

In [0]:
balanced_df = pd.DataFrame(grouped.apply(lambda x: x.sample(grouped.size().min()))).reset_index(drop=True)

In [0]:
balanced_df.domain.value_counts().nlargest(10).plot(kind='bar', color='goldenrod')
plt.xlabel('Domain')
plt.ylabel('Total Count')
plt.xticks(fontsize=14, rotation=90)
plt.title("'Domain' Counts in Undersampled Data", fontsize=25)
plt.savefig('domaincounts_under.png', bbox_inches='tight', dpi = 300)

In [0]:
balanced_df.domain.value_counts().sum()

We could also just undersample the two most overrepresented classes for domain, although this approach is less common.

In [0]:
nyt = df[df['domain'] == "nytimes.com"]

In [0]:
nyt_sample = nyt.sample(100)

In [0]:
mask = df['domain'].isin(['nytimes.com', 'beforeitsnews.com'])
newdf = df[~mask]

In [0]:
bef = df[df['domain'] == "beforeitsnews.com"]

In [0]:
bef_sample = bef.sample(100)

In [0]:
df_undersampling = pd.concat([newdf, nyt_sample, bef_sample], axis=0)

In [0]:
df_undersampling.domain.value_counts().nlargest(20).plot(kind='bar', color='grey')
plt.xlabel('Domain')
plt.ylabel('Total Count')
plt.xticks(fontsize=14, rotation=90)
plt.title("'Domain' Counts in Undersampled Data - Other Approach", fontsize=25)
plt.savefig('domaincounts_under2.png', bbox_inches='tight', dpi = 300)

Let's first base the analysis on the standard approach for undersampling. 

In [0]:
y = balanced_df['type']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(balanced_df['content'], y, test_size=0.33, random_state=53)



In [0]:
stopwords = stopwords.words('english')

In [0]:
for classifier in classifiers:
    countvect_pipeline = Pipeline([('preproc', pre_process()),
              ('cv', CountVectorizer(stop_words=stopwords)),
              ('clf',   classifier)])
    countvect_pipeline.fit(X_train,y_train)
    y_pred = countvect_pipeline.predict(X_test)
    print ("\n\nName of classifier being used with count vectorizer: \n\n" + str(classifier) + 
           "\n\nClassification Report : \n\n"+ str(metrics.classification_report(y_test,y_pred))  + "\n\n Accuracy of classifier : " + str(countvect_pipeline.score(X_test,y_test)))

In [0]:
for classifier in classifiers:
    tfidf_pipeline = Pipeline([
        #('preproc', pre_process()),
              ('tfidf', TfidfVectorizer(stop_words=stopwords)),
              ('clf',   classifier)])
    tfidf_pipeline.fit(X_train,y_train)
    y_pred = tfidf_pipeline.predict(X_test)
    print ("\n\nName of classifier being used with tfidf vectorizer: \n\n" + str(classifier) + 
           "\n\nClassification Report : \n\n"+ str(metrics.classification_report(y_test,y_pred))  + "\n\n Accuracy of classifier : " + str(tfidf_pipeline.score(X_test,y_test)))

In [0]:
for classifier in classifiers:
    tfidf_pipeline_bigrams = Pipeline([
        #('preproc', pre_process()),
              ('tfidf', TfidfVectorizer(ngram_range=(2,2),stop_words=stopwords)),
              ('clf',   classifier)])
    tfidf_pipeline_bigrams.fit(X_train,y_train)
    y_pred = tfidf_pipeline_bigrams.predict(X_test)
    print ("\n\nName of classifier being used with tfidf vectorizer: \n\n" + str(classifier) + 
           "\n\nClassification Report : \n\n"+ str(metrics.classification_report(y_test,y_pred))  + "\n\n Accuracy of classifier : " + str(tfidf_pipeline_bigrams.score(X_test,y_test)))

The most accurate model with tf-idf with linear SVC, with no additional preprocessing step.  Here is it without a pipeline to examine the most important features. 

In [0]:
tfidf_vect = TfidfVectorizer(stop_words=stopwords)

X_tfidf_train = tfidf_vect.fit_transform(X_train)

X_tfidf_test = tfidf_vect.transform(X_test)

In [0]:
linSVCclf = LinearSVC()

In [0]:
linSVCclf.fit(X_tfidf_train, y_train)
pred = linSVCclf.predict(X_tfidf_test)

score = metrics.accuracy_score(y_test, pred)
print(score)

In [0]:
show_most_informative_features(tfidf_vect, linSVCclf)

In [0]:
tfidf_vect_bigrams = TfidfVectorizer(ngram_range=(2,2),stop_words=stopwords)

In [0]:
X_tfidf__bigrams_train = tfidf_vect_bigrams.fit_transform(X_train)

In [0]:
X_tfidf__bigrams_test = tfidf_vect_bigrams.transform(X_test)

In [0]:
linSVCclf.fit(X_tfidf__bigrams_train, y_train)
pred = linSVCclf.predict(X_tfidf__bigrams_test)

In [0]:
score = metrics.accuracy_score(y_test, pred)
print(score)

In [0]:
show_most_informative_features(tfidf_vect_bigrams, nb_classifier)

Partially due to the amount of data, the informative features don't appear to be necessarily particularly informative.

Can unsupervised techniques, then, help us to glean insight from the dataset?

## Topic Modeling

Let's use the original sample, before the analysis considering domain, for topic modeling. 

In [0]:
df = pd.read_csv('data_large_new.csv',error_bad_lines=False, engine='python')

In [0]:
df.head()

In [0]:
df.shape

Clean the data and filter out non-English documents.

In [0]:
df = df[df['content'].notnull()]  


In [0]:
def lang_classify(row):
    lang = langid.classify(row)
    lang = lang[0]
    return lang

In [0]:
df['lang'] = df['content'].apply(lang_classify)


In [0]:
df.lang.value_counts()

In [0]:
df = df[df['lang']=='en']

In [0]:
df.shape

Include documents with the type labels 'fake' or 'reliable'.

In [0]:
df = df[(df['type']=='fake') | (df['type'] == 'reliable')]

In [0]:
df.type.value_counts()

In [0]:
reliable = df[(df['type']=='reliable')]

In [0]:
fake = df[(df['type']=='fake')]

Include the same number of documents labelled 'reliable' and 'fake'.

In [0]:
reliable = reliable.sample(n=24300, random_state=1)


In [0]:
fake = fake.sample(n=24300, random_state=1)



In [0]:
df = pd.concat([reliable, fake])

Further clean text.

In [0]:
punctuation = string.punctuation

In [0]:
english_stopwords = set(stopwords.words('english'))



In [0]:
ps = PorterStemmer()

In [0]:
def proc(row):
    text = row.lower()
    text = text.split()
    text =  [w for w in text if w.isalpha()]
    text = [w for w in text if w not in punctuation]
    text = [ps.stem(word) for word in text]
    text = [w for w in text if w not in english_stopwords]
    text = [w.encode('ascii', 'ignore').decode('ascii') for w in text]
    return text

In [0]:
df['content'] = df['content'].astype(str)

In [0]:
df['tokens_for_tm'] = df['content'].apply(proc)



Conduct topic modeling on all the documents. 

In [0]:
dictionary = corpora.Dictionary(df['tokens_for_tm'])

In [0]:
doc_term_matrix = [dictionary.doc2bow(item) for item in df["tokens_for_tm"]]

In [0]:
lsi = models.LsiModel(doc_term_matrix, num_topics=3, id2word=dictionary)

In [0]:
def plotTopicProjections(model,dictionary,scale=False,plotNegative=True,nTerms=6):
    '''
    Convenience function to plot term importances in topics
    @plotNegative is for models that return -ve term importances
    @scale is either fixed at [-1,1] or autoscaled based on largest importance
    @model is LDA/LSI gensim model object
    '''
    
    topicProjections=model.get_topics()
    
    for n in range(topicProjections.shape[0]):
        #print(i)
        topicTerm=collections.Counter({dictionary[j]:p for j,p in\
                                       enumerate(topicProjections[n,:])})

        most = topicTerm.most_common(nTerms)[::-1]
        least = topicTerm.most_common()[-1*nTerms:]
        
        if not scale:
            plt.xlim(-1,1)
            maxExtent=1
        else:
            maxMost=max([m[1] for m in most])*1.1
            minLeast=min([l[1] for l in least])*1.1
            
            maxMost=topicProjections.max()*1.1
            minMost=topicProjections.min()*1.1
            
            maxExtent=max([abs(minLeast),abs(maxMost)])
            plt.xlim(-1*maxExtent,maxExtent)
                    
        plt.barh(range(nTerms),[m[1] for m in most])
        for i,m in enumerate(most):
            plt.annotate('{:s} ({:.3f})'.format(m[0],m[1]),\
                         xy=(0.1*maxExtent,i-0.1),xycoords='data',fontsize=20)
        
        if not plotNegative:
            if not scale:
                plt.xlim(0,1)
            else:
                plt.xlim(0,maxExtent)
        
        plt.barh(range(nTerms),[l[1] for l in least])
        for i,l in enumerate(least):
            plt.annotate('{:s} ({:.3f})'.format(l[0],l[1]),\
                         xy=(-0.1*maxExtent,i-0.1),xycoords='data',ha='right',fontsize=20)
        plt.axvline(color='grey')
        plt.title('Topic {:d}'.format(n))
        plt.yticks([],[])
        plt.xlabel('Projection')
        plt.show()
        
# From Next-Generation-Natural-Language-Processing-with-Python 
# https://github.com/PacktPublishing/Next-Generation-Natural-Language-Processing-with-Python/blob/master/Section%205/utils.py

In [0]:
output = lsi.print_topics(num_topics=3, num_words = 6)
print (output)

In [0]:
plotTopicProjections(lsi,dictionary)

These topics are not very useful. Let's include more stopwords from the above results (and results from some other runs of topic modeling) and try again. 

In [0]:
english_stopwords = stopwords.words('english')



In [0]:
english_stopwords.extend(['wa', 'thi', 'hi', 'one','ha','ga','new','en','lo','que','el','said','ani','know','mani','peopl','like','would','even','becaus','use','rec','ap','de','ru','go','also','may','us','think','make','get','two','f'])


In [0]:
english_stopwords.extend(nltk.corpus.stopwords.words('spanish'))

In [0]:
df['tokens_for_tm'] = df['content'].apply(proc)

In [0]:
dictionary = corpora.Dictionary(df['tokens_for_tm'])

In addition to adding stopwords, we can also limit extremely common and rare words in the dictionary.

In [0]:
dictionary.filter_extremes(no_below=50, no_above=0.5)

In [0]:
doc_term_matrix = [dictionary.doc2bow(item) for item in df["tokens_for_tm"]]

In [0]:
lsi = models.LsiModel(doc_term_matrix, num_topics=7, id2word=dictionary)
output = lsi.print_topics(num_topics=7, num_words = 5)
print (output)

In [0]:
plotTopicProjections(lsi,dictionary)

Also use LDA, in addition to LSI: 

In [0]:
dictionary.filter_extremes(no_below=20, no_above=0.5)

In [0]:
lda = models.LdaModel(doc_term_matrix, num_topics=7, id2word=dictionary)

In [0]:
output = lda.print_topics(num_topics=3, num_words = 5)
print (output)

In [0]:
plotTopicProjections(lda,dictionary,plotNegative=False)

Several topics emerge from the sampled data, some more coherent than others. 

Let's perform topic modeling individually on articles labelled "fake" and articles labelled "reliable", to see if the topics are different for each set of labelled data:

In [0]:
fake.shape

In [0]:
fake['tokens_for_tm'] = fake['content'].apply(proc)

In [0]:
dictionary = corpora.Dictionary(fake["tokens_for_tm"])

In [0]:
dictionary.filter_extremes(no_below=20, no_above=0.5)

In [0]:
doc_term_matrix = [dictionary.doc2bow(item) for item in fake["tokens_for_tm"]]

In [0]:
lsi = models.LsiModel(doc_term_matrix, num_topics=7, id2word=dictionary)
output = lsi.print_topics(num_topics=7, num_words = 5)
print (output)

In [0]:
plotTopicProjections(lsi,dictionary)

Let's preform LDA:

In [0]:
lda = models.LdaModel(doc_term_matrix, num_topics=5, id2word=dictionary,passes=15, random_state=20)

output = lda.print_topics(num_topics=5, num_words = 5)
print (output)

In [0]:
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
pyLDAvis.gensim.prepare(lda, doc_term_matrix, dictionary)

Setting lambda to 0.2 in pyLDAvis yielded the most interesting results. Topic 1 is related to religion. Topic 2 concerns current events linked to war and foreign affairs. Topic 3 seems to be mostly associated with investments and economics. Topic 4 is associated American elections and politics. Topic 5 is the least coherent topic, which is associated with science, health, and technology.

<img src="viz-for-proj/fake/1.png">

<img src="viz-for-proj/fake/2.png">

<img src="viz-for-proj/fake/3.png">

<img src="viz-for-proj/fake/4.png">

<img src="viz-for-proj/fake/5.png">

For articles labelled "reliable":

In [0]:
reliable = df[df['type']=='reliable']

In [0]:
reliable.shape

In [0]:
reliable['tokens_for_tm'] = reliable['content'].apply(proc)

In [0]:
dictionary = corpora.Dictionary(reliable['tokens_for_tm'])
dictionary.filter_extremes(no_below=20, no_above=0.5)
doc_term_matrix = [dictionary.doc2bow(item) for item in reliable["tokens_for_tm"]]
lsi = models.LsiModel(doc_term_matrix, num_topics=5, id2word=dictionary)
output = lsi.print_topics(num_topics=5, num_words = 5)
print (output)

In [0]:
plotTopicProjections(lsi,dictionary)

Using LDA:

In [0]:
lda = models.LdaModel(doc_term_matrix, num_topics=5, id2word=dictionary, passes=15, random_state=20)

In [0]:
output = lda.print_topics(num_topics=5, num_words = 5)
print (output)

In [0]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, doc_term_matrix, dictionary)

As with the documents labelled "fake" lambda was set to 0.2 in pyLDAvis. Topic 1 seemed most associated with international politics and foreign affairs, with the most relevant terms being linked to war in the Middle East. Topic 2 included terms linked with church and religion. Topic 3 is linked to American politics. Topic 4 concerns current events in the Middle East. Topic 5 is associated with food and health.

<img src="viz-for-proj/real/1.png">

<img src="viz-for-proj/real/2.png">

<img src="viz-for-proj/real/3.png">

<img src="viz-for-proj/real/4.png">

<img src="viz-for-proj/real/5.png">

## multi-class classification

In [0]:
df = pd.read_csv('newly_sampled_data.csv')

In [0]:
df.type.value_counts().plot(kind='bar',color='springgreen')
plt.title("'Type' Counts in Initally Sampled Data", fontsize=25)

Take the top four categories to make a new df. 

In [0]:
fake_new_df = df[df['type']== 'fake'].sample(n=1000, random_state=0)

bias_new_df = df[df['type']== 'bias'].sample(n=1000, random_state=0)

political_new_df = df[df['type']== 'political'].sample(n=1000, random_state=0)

reliable_new_df = df[df['type']== 'reliable'].sample(n=1000, random_state=0)

new_df = pd.concat([fake_new_df, reliable_new_df,bias_new_df,political_new_df])

In [0]:
new_df.type.value_counts().plot(kind='bar',color='springgreen')
plt.title("New 'Type' Counts in Sampled Data", fontsize=25)

In [0]:
new_df.head()

y = new_df.type

X_train, X_test, y_train, y_test = train_test_split(new_df['content'], y, test_size=0.33, random_state=53)

In [0]:
for classifier in classifiers:
    pipeline_w_cv = Pipeline([
              ('cv', CountVectorizer(stop_words='english')),
              ('clf',   classifier)])
    pipeline_w_cv.fit(X_train,y_train)
    y_pred = pipeline_w_cv.predict(X_test)
    print ("\n\nName of classifier being used with count vectorizer: \n\n" + str(classifier) + 
           "\n\nClassification Report : \n\n"+ str(metrics.classification_report(y_test,y_pred))  + "\n\n Accuracy of classifier : " + str(pipeline_w_cv.score(X_test,y_test)))
    labels = ['fake','political','reliable','bias']
    cmtx = pd.DataFrame(
        metrics.confusion_matrix(y_test, y_pred,  labels=labels), 
        index=['true:fake', 'true:political','true:reliable','true:bias' ], 
        columns=['pred:fake', 'pred:political', 'pred:reliable', 'pred:bias'])
    print(cmtx)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cmtx,cmap='Oranges')
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    fig.colorbar(cax)
    plt.title('Confusion matrix of '+ str(classifier) + "\n")
    saveas = "cv_" + str(classifier)[0:7] +'.png'
    print (saveas)
    plt.savefig(saveas, bbox_inches='tight', dpi = 300)

In [0]:
for classifier in classifiers:
    pipeline_w_tfidf = Pipeline([
              ('tfidf', TfidfVectorizer(stop_words='english')),
              ('clf',   classifier)])
    pipeline_w_tfidf.fit(X_train,y_train)
    y_pred = pipeline_w_tfidf.predict(X_test)
    print ("\n\nName of classifier being used with tfidf vectorizer: \n\n" + str(classifier) + 
           "\n\nClassification Report : \n\n"+ str(metrics.classification_report(y_test,y_pred))  + "\n\n Accuracy of classifier : " + str(pipeline_w_tfidf.score(X_test,y_test)))
    labels = ['fake','political','reliable','bias']
    cmtx = pd.DataFrame(
        metrics.confusion_matrix(y_test, y_pred,  labels=labels), 
        index=['true:fake', 'true:political','true:reliable','true:bias' ], 
        columns=['pred:fake', 'pred:political', 'pred:reliable', 'pred:bias'])
    print(cmtx)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cmtx,cmap='Blues')
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    fig.colorbar(cax)
    plt.title('Confusion matrix of '+ str(classifier) + "\n")
    saveas = "tfidf_" + str(classifier)[0:5] +'.png'
    print (saveas)
    plt.savefig(saveas, bbox_inches='tight', dpi = 300)

### What happens when all of the original labels from the dataset are included in the classifier (not only 'fake','political','reliable','bias', but all of them)?

In [0]:
df = pd.read_csv('newly_sampled_data.csv')

In [0]:
df.shape

In [0]:
df = df[pd.notnull(df['content'])]

In [0]:
df = df[pd.notnull(df['type'])]

In [0]:
df.shape

In [0]:
df.type.value_counts().plot(kind='bar',color='springgreen')
plt.title("'Type' Counts in Initally Sampled Data", fontsize=25)

In [0]:
df.type.value_counts()

In [0]:
y = df.type

X_train, X_test, y_train, y_test = train_test_split(df['content'], y, test_size=0.20, random_state=53)

In [0]:
for classifier in classifiers:
    pipeline_w_cv = Pipeline([
              ('cv', CountVectorizer(stop_words='english')),
              ('clf',   classifier)])
    pipeline_w_cv.fit(X_train,y_train)
    y_pred = pipeline_w_cv.predict(X_test)
    print ("\n\nName of classifier being used with count vectorizer: \n\n" + str(classifier) + 
           "\n\nClassification Report : \n\n"+ str(metrics.classification_report(y_test,y_pred))  + "\n\n Accuracy of classifier : " + str(pipeline_w_cv.score(X_test,y_test)))
    labels = ['reliable', 'political','bias','fake','conspiracy','rumor','unknown','unreliable', 'clickbait','junksci','satire','hate']
    cmtx = pd.DataFrame(
    metrics.confusion_matrix(y_test, y_pred,  labels=labels), 
    index=['true:reliable', 'true:political','true:bias','true:fake','true:conspiracy','true:rumor','true:unknown','true:unreliable', 'true:clickbait','true:junksci','true:satire','true:hate'], 
    columns=['pred:reliable', 'pred:political', 'pred:bias', 'pred:fake','pred:conspiracy','pred:rumor','pred:unknown','pred:unreliable','pred:clickbait','pred:junksci','pred:satire','pred:hate']
)
    print(cmtx)
    

    
    title = "cv_" + str(classifier)[0:10]
    
    
    
    ax = sns.heatmap(cmtx, cmap="YlGnBu")
    
    
    ax.set_title(title)

    saveas = "all_labels_cv_" + str(classifier)[0:5] +'.png'
    plt.show()
    ax.figure.savefig(saveas, bbox_inches='tight', dpi = 300)
    plt.clf()


 

In [0]:
for classifier in classifiers:
    pipeline_w_tfidf = Pipeline([
              ('tfidf', TfidfVectorizer()),
              ('clf',   classifier)])
    pipeline_w_tfidf.fit(X_train,y_train)
    y_pred = pipeline_w_tfidf.predict(X_test)
    print ("\n\nName of classifier being used with tfidf vectorizer: \n\n" + str(classifier) + 
           "\n\nClassification Report : \n\n"+ str(metrics.classification_report(y_test,y_pred))  + "\n\n Accuracy of classifier : " + str(pipeline_w_tfidf.score(X_test,y_test)))
    cmtx = pd.DataFrame(
    metrics.confusion_matrix(y_test, y_pred,  labels=labels), 
    index=['true:reliable', 'true:political','true:bias','true:fake','true:conspiracy','true:rumor','true:unknown','true:unreliable', 'true:clickbait','true:junksci','true:satire','true:hate'], 
    columns=['pred:reliable', 'pred:political', 'pred:bias', 'pred:fake','pred:conspiracy','pred:rumor','pred:unknown','pred:unreliable','pred:clickbait','pred:junksci','pred:satire','pred:hate']
)
    print(cmtx)
    
    
    
    title = "tfidf_" + str(classifier)[0:10]
    
    
    
    ax = sns.heatmap(cmtx, cmap="Purples")
    ax.set_title(title)
    
    
    
    plt.show()
    saveas = "all_labels_tfidf_" + str(classifier)[0:5] +'.png'
    ax.figure.savefig(saveas, bbox_inches='tight', dpi = 300)
    plt.clf()
 